<a href="https://colab.research.google.com/github/monta0315/DGM-and-VR/blob/main/VR/fine_tuning_pra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn,optim
from torchvision import datasets,transforms,models
import numpy as np
import tqdm

torchvisionはtorchのライブラリの集合体
transformsは一般的な画像変換を利用する際に使用するメソッド
Composeを利用して画像変換を連結することができる

In [ ]:
#学習済モデルでは前処理に以下の平均分散を用いた正規化を行う
normalize = transforms.Normalize(mean=[0.485,0.456,0.408],std=[0.229,0.224,0.225])

#データセットの定義に用いるtransformを定義する
#Big Transferを参考にサイズを128にリサイズする(もともとは32)
#validデータを利用する理由として訓練後の分類器のハイパーパラメータの調整をする
transform_valid = transforms.Compose([
                                      transforms.Resize(128),
                                      transforms.ToTensor(),
                                      normalize
])

#訓練データではデータAugumentationも加える
#Data Augumentationとは => 過学習を防ぐためやそもそもの学習データ数が足りない場合のために既存の学習データを増やす => 今回はrandomcropをしているところがData Augumentation
#randomCropは一枚の画像からランダムに切り抜くこと->多分引数は画像サイズ
#randomhorizontalflipはランダムに左右反転を行う=>これもDataAugumentationの一環
transform_train = transforms.Compose([
                                      transforms.Resize(160),
                                      transforms.RandomCrop(128),
                                      transforms.RandomHorizontalFlip(0.5),
                                      transforms.ToTensor(),
                                      normalize
])

batch_size = 128

#CIFAR10の50000枚の訓練データを分割し、5000枚のみを用いる
train_dataset = datasets.CIFAR10('./data/cifar10',train=True,download=True,transform=transform_train)
train_dataset, _ = torch.utils.data.random_split(train_dataset,[5000,45000])

dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True
)

valid_dataset = datasets.CIFAR10('./data/cifar10',train=False,download=True,transform=transform_valid)
valid_dataset,_ = torch.utils.data.random_split(valid_dataset,[3000,7000])

dataloader_valid = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=True
)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10
Files already downloaded and verified


##ファインチューニングを行わない場合
まずはベースラインとしてファインチューニングを行わない場合の精度を確認する
モデルにはResNet18を用いる→なんでも良い

In [ ]:
n_epochs = 10
lr = 0.001
device = 'cuda'

# pytorchではtorchvision.modelsから有名なモデルを呼び出してそのまま用いることが出来る
resnet18 = models.resnet18()

# デフォルトのresnet18は1000クラス分類用なので、全結合層を10クラス用に変更する
resnet18.fc = nn.Linear(512,10)

resnet18.to(device)
optimizer = optim.Adam(resnet18.parameters(), lr=lr)

torchの学習フェーズ
基本forの2重ループ

tqdmはプログレスバーを表示させる

In [ ]:
for epoch in range(n_epochs):
  losses_train = []
  losses_valid = []

  resnet18.train()
  n_train = 0
  acc_train = 0
  for x,t in tqdm.notebook.tqdm(dataloader_train):
    n_train += t.size()[0]

    resnet18.zero_grad() #勾配の初期化

    x = x.to(device) #テンソルをGPUに移動

    t_hot = torch.eye(10)[t] #正解ラベルをone-hot vector化

    t= t.to(device)
    t_hot = t_hot.to(device) #正解ラベルとone-hto vectorをそれぞれGPUに移動

    y = resnet18(x) #順伝播

    loss = -(t_hot*torch.log_softmax(y,dim=-1)).sum(axis=-1).mean() #誤差（クロスエントロピーごさ関数）の計算

    loss.backward() #誤差の逆伝播

    optimizer.step() #パラメータの更新

    pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

    acc_train += (pred==t).float().sum().item()
    losses_train.append(loss.tolist())
  
  resnet18.eval()
  n_val = 0
  acc_val = 0
  for x,t in dataloader_valid:
    n_val += t.size()[0]

    x = x.to(device)  # テンソルをGPUに移動

    t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

    t = t.to(device)
        
    t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

    y = resnet18(x)  # 順伝播

    loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

    pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

    acc_val += (pred == t).float().sum().item()
    losses_valid.append(loss.tolist())

print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

EPOCH: 9, Train [Loss: 1.156, Accuracy: 0.590], Valid [Loss: 1.280, Accuracy: 0.550]


##学習済モデルの使用

次は学習済みモデルを用いる。

pytorchではtorchvisionのデフォルトのモデルに対して、ImageNetの学習済みモデルを用いることが出来る。

ImageNetは1000クラス約130万枚のデータセットで、CIFAR10よりもはるかに大規模なデータセットである。

In [ ]:
n_epochs = 10
lr = 0.001
device = 'cuda'

# pretrained = Trueと入れるとImageNetで事前学習された重みがセットされる
resnet18_pretrained = models.resnet18(pretrained=True)

resnet18_pretrained.fc = nn.Linear(512,10)

resnet18_pretrained.to(device)
optimizer = optim.Adam(resnet18_pretrained.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    resnet18_pretrained.train()
    n_train = 0
    acc_train = 0
    for x, t in tqdm.notebook.tqdm(dataloader_train):
        n_train += t.size()[0]

        resnet18_pretrained.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18_pretrained(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    resnet18_pretrained.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動

        t_hot = torch.eye(10)[t]  # 正解ラベルをone-hot vector化

        t = t.to(device)
        t_hot = t_hot.to(device)  # 正解ラベルとone-hot vectorをそれぞれGPUに移動

        y = resnet18_pretrained(x)  # 順伝播

        loss = -(t_hot*torch.log_softmax(y, dim=-1)).sum(axis=1).mean()  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

  0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


EPOCH: 0, Train [Loss: 1.081, Accuracy: 0.653], Valid [Loss: 1.278, Accuracy: 0.635]


  0%|          | 0/40 [00:00<?, ?it/s]

EPOCH: 1, Train [Loss: 0.684, Accuracy: 0.761], Valid [Loss: 1.230, Accuracy: 0.636]


...

EPOCH: 7, Train [Loss: 0.368, Accuracy: 0.890], Valid [Loss: 0.855, Accuracy: 0.762]


  0%|          | 0/40 [00:00<?, ?it/s]

EPOCH: 8, Train [Loss: 0.632, Accuracy: 0.801], Valid [Loss: 0.787, Accuracy: 0.752]


  0%|          | 0/40 [00:00<?, ?it/s]

EPOCH: 9, Train [Loss: 0.482, Accuracy: 0.852], Valid [Loss: 0.639, Accuracy: 0.791]
